Khi tách cấu trúc dự án thành **backend** và **frontend**, chúng ta sẽ tổ chức theo cách sau:

---

### **Cấu trúc dự án**
```bash
project/
├── backend/                       # Code xử lý server-side (API, model)
│   ├── app.py                     # Flask app (backend API)
│   ├── model/
│   │   └── best.pt                # Model YOLO đã huấn luyện
│   ├── uploads/                   # Thư mục chứa ảnh upload và kết quả
│   └── requirements.txt           # Thư viện cần thiết cho backend
├── frontend/                      # Code xử lý client-side (UI, static files)
│   ├── index.html                 # Trang chính giao diện người dùng
│   ├── static/
│   │   ├── style.css              # File CSS cho UI
│   │   └── script.js              # File JavaScript xử lý logic
├── README.md                      # Hướng dẫn sử dụng
```

---

### **Chi tiết từng phần**

#### **1. Backend**
**File `backend/app.py`**:
```python
from flask import Flask, request, jsonify, send_from_directory
import os
import cv2
from ultralytics import YOLO
import numpy as np

app = Flask(__name__)

# Cấu hình
MODEL_PATH = "model/best.pt"      # Đường dẫn model YOLO
UPLOAD_FOLDER = "uploads"         # Thư mục chứa ảnh
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Load model
model = YOLO(MODEL_PATH)

@app.route("/upload", methods=["POST"])
def upload():
    # Kiểm tra file upload
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded!"}), 400

    file = request.files["file"]
    if file.filename == "":
        return jsonify({"error": "No selected file!"}), 400

    file_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(file_path)

    # Chạy YOLO inference
    results = model(file_path, conf=0.3)
    output_image = results[0].plot()

    # Lưu ảnh kết quả
    output_path = os.path.join(UPLOAD_FOLDER, f"output_{file.filename}")
    cv2.imwrite(output_path, cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR))

    return jsonify({"output_url": f"/uploads/output_{file.filename}"})

@app.route("/uploads/<filename>")
def uploaded_file(filename):
    return send_from_directory(UPLOAD_FOLDER, filename)

if __name__ == "__main__":
    app.run(debug=True)
```

**File `backend/requirements.txt`**:
```plaintext
flask
ultralytics
opencv-python
numpy
```

---

#### **2. Frontend**
**File `frontend/index.html`**:
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>YOLOv8 Detection</title>
    <link rel="stylesheet" href="static/style.css">
</head>
<body>
    <div class="container">
        <h1>Object Detection with YOLOv8</h1>
        <form id="upload-form">
            <input type="file" name="file" id="file" accept="image/*">
            <button type="submit">Upload and Detect</button>
        </form>
        <div id="result">
            <img id="output-image" src="" alt="Result will be shown here">
        </div>
    </div>
    <script src="static/script.js"></script>
</body>
</html>
```

**File `frontend/static/style.css`**:
```css
body {
    font-family: Arial, sans-serif;
    text-align: center;
    background-color: #f9f9f9;
}

.container {
    max-width: 600px;
    margin: 50px auto;
    padding: 20px;
    background: white;
    box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
}

h1 {
    color: #333;
}

form {
    margin: 20px 0;
}

#result img {
    margin-top: 20px;
    max-width: 100%;
    height: auto;
    border: 2px solid #ddd;
    border-radius: 5px;
}
```

**File `frontend/static/script.js`**:
```javascript
document.getElementById("upload-form").addEventListener("submit", async function (event) {
    event.preventDefault();

    const fileInput = document.getElementById("file");
    const file = fileInput.files[0];
    if (!file) {
        alert("Please select an image to upload!");
        return;
    }

    const formData = new FormData();
    formData.append("file", file);

    try {
        const response = await fetch("http://127.0.0.1:5000/upload", {
            method: "POST",
            body: formData,
        });

        if (response.ok) {
            const data = await response.json();
            const outputImage = document.getElementById("output-image");
            outputImage.src = "http://127.0.0.1:5000" + data.output_url;
        } else {
            alert("Failed to process the image!");
        }
    } catch (error) {
        alert("An error occurred: " + error.message);
    }
});
```

---

### **3. Cách chạy ứng dụng**

#### **1. Khởi động backend**
```bash
cd backend
pip install -r requirements.txt
python app.py
```

#### **2. Mở frontend**
- Chỉ cần mở `frontend/index.html` trong trình duyệt.
- Đảm bảo backend đang chạy trên `http://127.0.0.1:5000`.

---

### **4. Triển khai**
- **Frontend**: 
  - Đưa lên dịch vụ tĩnh như **Netlify** hoặc **Vercel**.
- **Backend**: 
  - Deploy lên cloud như **Heroku**, **Render**, hoặc **AWS EC2**.
- **Kết nối**: 
  - Đảm bảo chỉnh sửa URL API trong `script.js` để trỏ đến backend đã deploy.

Nếu cần hỗ trợ triển khai hoặc mở rộng, hãy cho mình biết nhé! 🚀